In [41]:
# -*- coding: UTF-8 -*-
import random
from PIL import Image, ImageDraw
import numpy as np
import time
from progressbar import *
import pickle
import os
import sys
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
# basePath = os.path.abspath('.')
# pyFilePath = os.path.dirname(sys.argv[0])
pyFilePath = "/Users/zac/5-Algrithm/python"
img_target = Image.open(pyFilePath+"/genetic_img/timg.jpeg")
img_savePath = pyFilePath+"/genetic_img"

In [133]:
class Scollop(object):
    @staticmethod
    def generate_triangle(img, width, height):
        point_a = (random.randint(0, width), random.randint(0, height))
        point_b = (random.randint(0, width), random.randint(0, height))
        point_c = (random.randint(0, width), random.randint(0, height))
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        drw = ImageDraw.Draw(img, 'RGBA')
        drw.polygon([point_a, point_b, point_c], color)
        return point_a, point_b, point_c, color

    def ancestor(self):
        for _ in range(self.gene_count):
            self.positions.append(self.generate_triangle(self.img, self.size[0], self.size[1]))

    def __init__(self, gene_count, size, positions=list()):
        self.gene_count = gene_count
        self.positions = positions
        self.size = size
        self.img = Image.new('RGB', self.size, color=(255, 255, 255))
        self.loss = 0.0

In [126]:
print("begin")
# 初始化100000个
herd_size = 5000
# 迭代80w代
r_generation = 300000
# 每次迭代淘汰 1000个,随着迭代进行,每进行 2000轮淘汰个数减少100个,淘汰个数最少 200
weed_out = 200
weed_update_lock = False
epoch_weed = 3000
min_weed_out = 50
# 变异率初期小,后期迭代进行变异率增加,最多0.6
varation_init_rate_global = 0.1
varation_update_lock = True
epoch_varation = 3000
varation_update_rate = 0.05
max_varation_rate_global = 0.5

begin


In [9]:
# 生成一个子代的方法
def copulation(scollop, scollop_another, varation_rate, max_varation_rate = 0.9):
    new_position = random.sample(scollop.positions, int(scollop.gene_count * 0.5)) + random.sample(scollop_another.positions,int(scollop_another.gene_count * 0.5))
    new_scollop = Scollop(gene_count=len(new_position), size=scollop.size, positions=new_position)
    img = Image.new('RGB', scollop.size, color=(255, 255, 255))
    drw = ImageDraw.Draw(img, 'RGBA')
    varation_rate_t = max_varation_rate if varation_rate>= max_varation_rate else varation_rate
    for p in new_position:
        if random.random()<=varation_rate_t:
            new_scollop.generate_triangle(img,scollop.size[0],scollop.size[1])
        else:
            drw.polygon([p[0], p[1], p[2]], p[3])
    new_scollop.img = img
    return new_scollop

In [10]:
# loss损失函数
def calculate_loss(image1,image2):
    img_1 = np.asarray(image1, dtype="int32")
    img_2 = np.asarray(image2, dtype="int32")
    # loss_result = np.count_nonzero(img_1 - img_2)
    # loss_result = np.dot(img_1.flatten(),img_2.flatten())/(np.sqrt(sum(img_1.flatten()*img_1.flatten()))*np.sqrt(sum(img_2.flatten()*img_2.flatten())))
    # loss_result = sum(map(sum, np.nonzero(img_1 - img_2)))
    loss_result = 1.0*np.abs(img_1-img_2).sum()/np.abs(img_1-img_2).size
    return loss_result

In [101]:
# 淘汰方式
def select_func(generation_sample, topN):
    # 直接进行末尾淘汰
    selected = sorted(generation_sample, key=lambda x: x[1])[:topN]
    return selected

In [102]:
def sub_generation(generation_sample, varation_rate,weed_cnt):
        born = list()
        for _ in range(int(len(generation_sample)*0.5)):
            s1,s2=map(lambda x:x[0], random.sample(generation_sample,2))
            # 迭代生成新的Scollop
            s_new = copulation(s1,s2,varation_rate,max_varation_rate=max_varation_rate_global)
            # 计算loss
            loss = calculate_loss(s_new.img, img_target)
            # 合并
            born.append((s_new,loss))
        # 子代取最优的n个
        topN = select_func(born,weed_cnt)
        return topN

In [100]:
def sub_generation_v2(generation_sample,varation_rate,weed_cnt):
        # 挑选 topN 个互相杂交, 再全局随机挑选 n个互相杂交,最后得到n个子代
        topN_cop = select_func(generation_sample,weed_cnt)
        top_born = list()
        for _ in range(int(len(topN_cop)*0.5)):
            s1,s2=map(lambda x:x[0], random.sample(topN_cop,2))
            # 迭代生成新的Scollop
            s_new = copulation(s1,s2,varation_rate,max_varation_rate=max_varation_rate_global)
            # 计算loss
            loss = calculate_loss(s_new.img, img_target)
            # 合并
            top_born.append((s_new,loss))
        random_cop = random.sample(generation_sample,weed_cnt)
        random_born = list()
        for _ in range(int(len(random_cop)*0.5)):
            s1,s2=map(lambda x:x[0], random.sample(random_cop,2))
            # 迭代生成新的Scollop
            s_new = copulation(s1,s2,varation_rate,max_varation_rate=max_varation_rate_global)
            # 计算loss
            loss = calculate_loss(s_new.img, img_target)
            # 合并
            random_born.append((s_new,loss))
        return select_func(top_born+random_born,weed_cnt)

In [131]:
def dynamic_varation(v, cnt):
    if cnt >= 20 and varation_update_lock:
        # 即只有出现第n轮和第n+100轮的最优解相同,变异率增加 0.05
        print("cnt is %s" %cnt)
        print("varation change from %s" % v)
        v = v+varation_update_rate if v<=max_varation_rate_global else max_varation_rate_global
        print("cnt is %s, varation change to %s" %(cnt,v))
        sys.stdout.flush()
    else:
        # 如果未开启自适应变异 || cnt重置为1了(表示有新的最优解了) => 变异率重置
        v = varation_init_rate_global
    return v

In [123]:
# 初始化种群
ancestor_sample = list()
pbar_ancestor = ProgressBar(widgets=[Percentage(),Bar(),' ', Timer(),], maxval=herd_size).start()
for i in range(herd_size):
    s = Scollop(100, img_target.size)
    s.ancestor()
    # 计算loss
    ancestor_loss = calculate_loss(s.img, img_target)
    # 合并
    ancestor_sample.append((s, ancestor_loss))
    pbar_ancestor.update(i+1)
pbar_ancestor.finish()
print ("ancestor done!")

100%|###################################################| Elapsed Time: 0:00:26


ancestor done!


In [135]:
ancestor_sample_read=ancestor_sample
sample_selected = list()
# 记录一下有多少代的最优解相同,100代大概是9min
cnt_of_same_best = 1
# 初始化变异率
varation_input = varation_init_rate_global

In [136]:
for generation in range(r_generation):
        time_start = time.time()
        # 自然选择
        sample_selected = select_func(ancestor_sample_read, herd_size - weed_out)
        # 每100代展示一次最优结果
        if generation % 20 == 0:
            print ("generation: %s, best loss: %s" % (generation, sample_selected[0][1]))
        if generation % 100 == 0:
            print("    generation: %s, worst loss: %s"% (generation, sample_selected[-1][1]))
        # 每100代保存一次结果
        if generation % 100 == 0:
            sample_selected[0][0].img.save("%s/%s_%s.jpg" % (img_savePath, generation, sample_selected[0][1]))
        # 变异率
        varation_input = dynamic_varation(v=varation_input,cnt=cnt_of_same_best)
        # 生成下一代
        top_of_new_born = sub_generation_v2(sample_selected, varation_input, weed_out)
        # 子代和父代合并
        ancestor_sample_read = sample_selected + top_of_new_born
        # 如果子代中没有产生更优解,说明这一轮后的最优解没有变化
        print("    (%s, %s)" %(sample_selected[0][1],top_of_new_born[0][1]))
        if top_of_new_born[0][1] >= sample_selected[0][1]:
            cnt_of_same_best += 1
            print("    >>>cnt:%s, v:%s" % (cnt_of_same_best,varation_input))
        else:
            cnt_of_same_best = 1
        time_end = time.time()

generation: 0, best loss: 63.2871440329
    generation: 0, worst loss: 84.7445102881
    (63.2871440329, 64.7255473251)
    >>>cnt:2, v:0.1
    (63.2871440329, 67.765399177)
    >>>cnt:3, v:0.1
    (63.2871440329, 65.951654321)
    >>>cnt:4, v:0.1
    (63.2871440329, 64.2402962963)
    >>>cnt:5, v:0.1
    (63.2871440329, 60.601563786)
    (60.601563786, 65.4581234568)
    >>>cnt:2, v:0.1
    (60.601563786, 65.5850205761)
    >>>cnt:3, v:0.1
    (60.601563786, 63.6607078189)
    >>>cnt:4, v:0.1
    (60.601563786, 65.9723786008)
    >>>cnt:5, v:0.1
    (60.601563786, 64.8133497942)
    >>>cnt:6, v:0.1
    (60.601563786, 63.4539423868)
    >>>cnt:7, v:0.1
    (60.601563786, 66.7239835391)
    >>>cnt:8, v:0.1
    (60.601563786, 63.6184526749)
    >>>cnt:9, v:0.1
    (60.601563786, 61.6064197531)
    >>>cnt:10, v:0.1
    (60.601563786, 64.343308642)
    >>>cnt:11, v:0.1
    (60.601563786, 66.7050864198)
    >>>cnt:12, v:0.1
    (60.601563786, 62.9534650206)
    >>>cnt:13, v:0.1
    (60.6015

    (47.2447572016, 50.0717530864)
    >>>cnt:44, v:0.5
cnt is 44
varation change from 0.5
cnt is 44, varation change to 0.55
    (47.2447572016, 52.8310123457)
    >>>cnt:45, v:0.55
cnt is 45
varation change from 0.55
cnt is 45, varation change to 0.5
    (47.2447572016, 51.1627654321)
    >>>cnt:46, v:0.5
cnt is 46
varation change from 0.5
cnt is 46, varation change to 0.55
    (47.2447572016, 53.0240329218)
    >>>cnt:47, v:0.55
cnt is 47
varation change from 0.55
cnt is 47, varation change to 0.5
    (47.2447572016, 51.8200658436)
    >>>cnt:48, v:0.5
generation: 120, best loss: 47.2447572016
cnt is 48
varation change from 0.5
cnt is 48, varation change to 0.55
    (47.2447572016, 53.5854485597)
    >>>cnt:49, v:0.55
cnt is 49
varation change from 0.55
cnt is 49, varation change to 0.5
    (47.2447572016, 53.2500576132)
    >>>cnt:50, v:0.5
cnt is 50
varation change from 0.5
cnt is 50, varation change to 0.55
    (47.2447572016, 48.4086748971)
    >>>cnt:51, v:0.55
cnt is 51
varati

    (43.7691851852, 48.5215967078)
    >>>cnt:59, v:0.55
cnt is 59
varation change from 0.55
cnt is 59, varation change to 0.5
    (43.7691851852, 48.3211687243)
    >>>cnt:60, v:0.5
cnt is 60
varation change from 0.5
cnt is 60, varation change to 0.55
    (43.7691851852, 50.9652674897)
    >>>cnt:61, v:0.55
cnt is 61
varation change from 0.55
cnt is 61, varation change to 0.5
    (43.7691851852, 51.3503868313)
    >>>cnt:62, v:0.5
cnt is 62
varation change from 0.5
cnt is 62, varation change to 0.55
    (43.7691851852, 51.2231769547)
    >>>cnt:63, v:0.55
cnt is 63
varation change from 0.55
cnt is 63, varation change to 0.5
    (43.7691851852, 51.5688230453)
    >>>cnt:64, v:0.5
cnt is 64
varation change from 0.5
cnt is 64, varation change to 0.55
    (43.7691851852, 50.3791934156)
    >>>cnt:65, v:0.55
cnt is 65
varation change from 0.55
cnt is 65, varation change to 0.5
    (43.7691851852, 52.160691358)
    >>>cnt:66, v:0.5
cnt is 66
varation change from 0.5
cnt is 66, varation chan

    (43.7691851852, 51.6647572016)
    >>>cnt:122, v:0.5
cnt is 122
varation change from 0.5
cnt is 122, varation change to 0.55
    (43.7691851852, 48.8782222222)
    >>>cnt:123, v:0.55
cnt is 123
varation change from 0.55
cnt is 123, varation change to 0.5
    (43.7691851852, 51.1942880658)
    >>>cnt:124, v:0.5
cnt is 124
varation change from 0.5
cnt is 124, varation change to 0.55
    (43.7691851852, 52.5916213992)
    >>>cnt:125, v:0.55
cnt is 125
varation change from 0.55
cnt is 125, varation change to 0.5
    (43.7691851852, 51.3750617284)
    >>>cnt:126, v:0.5
cnt is 126
varation change from 0.5
cnt is 126, varation change to 0.55
    (43.7691851852, 49.4843950617)
    >>>cnt:127, v:0.55
cnt is 127
varation change from 0.55
cnt is 127, varation change to 0.5
    (43.7691851852, 51.997382716)
    >>>cnt:128, v:0.5
generation: 260, best loss: 43.7691851852
cnt is 128
varation change from 0.5
cnt is 128, varation change to 0.55
    (43.7691851852, 50.1542880658)
    >>>cnt:129, v:

    (43.7691851852, 48.330617284)
    >>>cnt:185, v:0.55
cnt is 185
varation change from 0.55
cnt is 185, varation change to 0.5
    (43.7691851852, 51.0521481481)
    >>>cnt:186, v:0.5
cnt is 186
varation change from 0.5
cnt is 186, varation change to 0.55
    (43.7691851852, 51.394962963)
    >>>cnt:187, v:0.55
cnt is 187
varation change from 0.55
cnt is 187, varation change to 0.5
    (43.7691851852, 46.9)
    >>>cnt:188, v:0.5
generation: 320, best loss: 43.7691851852
cnt is 188
varation change from 0.5
cnt is 188, varation change to 0.55
    (43.7691851852, 49.8073251029)
    >>>cnt:189, v:0.55
cnt is 189
varation change from 0.55
cnt is 189, varation change to 0.5
    (43.7691851852, 51.4863539095)
    >>>cnt:190, v:0.5
cnt is 190
varation change from 0.5
cnt is 190, varation change to 0.55
    (43.7691851852, 50.3742386831)
    >>>cnt:191, v:0.55
cnt is 191
varation change from 0.55
cnt is 191, varation change to 0.5
    (43.7691851852, 50.3189958848)
    >>>cnt:192, v:0.5
cnt i

...

    (43.7691851852, 49.9856790123)
    >>>cnt:435, v:0.55
cnt is 435
varation change from 0.55
cnt is 435, varation change to 0.5
    (43.7691851852, 50.933744856)
    >>>cnt:436, v:0.5
cnt is 436
varation change from 0.5
cnt is 436, varation change to 0.55
    (43.7691851852, 46.820691358)
    >>>cnt:437, v:0.55
cnt is 437
varation change from 0.55
cnt is 437, varation change to 0.5
    (43.7691851852, 49.8380411523)
    >>>cnt:438, v:0.5
cnt is 438
varation change from 0.5
cnt is 438, varation change to 0.55
    (43.7691851852, 47.3774156379)
    >>>cnt:439, v:0.55
cnt is 439
varation change from 0.55
cnt is 439, varation change to 0.5
    (43.7691851852, 52.2354567901)
    >>>cnt:440, v:0.5
cnt is 440
varation change from 0.5
cnt is 440, varation change to 0.55
    (43.7691851852, 49.6872098765)
    >>>cnt:441, v:0.55
cnt is 441
varation change from 0.55
cnt is 441, varation change to 0.5
    (43.7691851852, 50.8619423868)
    >>>cnt:442, v:0.5
cnt is 442
varation change from 0.5
cn

    (43.7691851852, 45.6410699588)
    >>>cnt:497, v:0.55
cnt is 497
varation change from 0.55
cnt is 497, varation change to 0.5
    (43.7691851852, 47.7548971193)
    >>>cnt:498, v:0.5
cnt is 498
varation change from 0.5
cnt is 498, varation change to 0.55
    (43.7691851852, 51.2181069959)
    >>>cnt:499, v:0.55
cnt is 499
varation change from 0.55
cnt is 499, varation change to 0.5
    (43.7691851852, 49.0022222222)
    >>>cnt:500, v:0.5
cnt is 500
varation change from 0.5
cnt is 500, varation change to 0.55
    (43.7691851852, 49.8146831276)
    >>>cnt:501, v:0.55
cnt is 501
varation change from 0.55
cnt is 501, varation change to 0.5
    (43.7691851852, 48.1058765432)
    >>>cnt:502, v:0.5
cnt is 502
varation change from 0.5
cnt is 502, varation change to 0.55
    (43.7691851852, 47.9236707819)
    >>>cnt:503, v:0.55
cnt is 503
varation change from 0.55
cnt is 503, varation change to 0.5
    (43.7691851852, 48.6603621399)
    >>>cnt:504, v:0.5
cnt is 504
varation change from 0.5


    (42.1469135802, 43.5583703704)
    >>>cnt:7, v:0.1
    (42.1469135802, 41.8997530864)
    (41.8997530864, 43.0449876543)
    >>>cnt:2, v:0.1
    (41.8997530864, 43.5982716049)
    >>>cnt:3, v:0.1
    (41.8997530864, 41.7525596708)
    (41.7525596708, 43.3379588477)
    >>>cnt:2, v:0.1
    (41.7525596708, 42.6320493827)
    >>>cnt:3, v:0.1
generation: 720, best loss: 41.7525596708
    (41.7525596708, 42.9617119342)
    >>>cnt:4, v:0.1
    (41.7525596708, 41.2589465021)
    (41.2589465021, 42.2666337449)
    >>>cnt:2, v:0.1
    (41.2589465021, 43.5836872428)
    >>>cnt:3, v:0.1
    (41.2589465021, 43.8315390947)
    >>>cnt:4, v:0.1
    (41.2589465021, 41.6456460905)
    >>>cnt:5, v:0.1
    (41.2589465021, 42.7244444444)
    >>>cnt:6, v:0.1
    (41.2589465021, 43.6809876543)
    >>>cnt:7, v:0.1
    (41.2589465021, 42.72381893)
    >>>cnt:8, v:0.1
    (41.2589465021, 42.1917530864)
    >>>cnt:9, v:0.1
    (41.2589465021, 42.9474074074)
    >>>cnt:10, v:0.1
    (41.2589465021, 43.922386

    (41.2589465021, 46.4964444444)
    >>>cnt:71, v:0.55
cnt is 71
varation change from 0.55
cnt is 71, varation change to 0.5
    (41.2589465021, 46.1562469136)
    >>>cnt:72, v:0.5
cnt is 72
varation change from 0.5
cnt is 72, varation change to 0.55
    (41.2589465021, 47.9573333333)
    >>>cnt:73, v:0.55
cnt is 73
varation change from 0.55
cnt is 73, varation change to 0.5
    (41.2589465021, 46.7263868313)
    >>>cnt:74, v:0.5
cnt is 74
varation change from 0.5
cnt is 74, varation change to 0.55
    (41.2589465021, 47.8503045267)
    >>>cnt:75, v:0.55
cnt is 75
varation change from 0.55
cnt is 75, varation change to 0.5
    (41.2589465021, 49.2534485597)
    >>>cnt:76, v:0.5
cnt is 76
varation change from 0.5
cnt is 76, varation change to 0.55
    (41.2589465021, 47.8110946502)
    >>>cnt:77, v:0.55
cnt is 77
varation change from 0.55
cnt is 77, varation change to 0.5
    (41.2589465021, 45.4010205761)
    >>>cnt:78, v:0.5
cnt is 78
varation change from 0.5
cnt is 78, varation cha

KeyboardInterrupt: 